In [ ]:
import pandas as pd 
import json
import numpy as np
from sklearn.metrics import precision_recall_fscore_support



In [ ]:
from transformers import AutoTokenizer, pipeline
from custom_dataset import LedgerDataset

In [ ]:
file_name = 'testing.xlsx'
save_json_name ='test.json'
model_name = "output_base_3/checkpoint-10000/"


In [ ]:
df = pd.read_excel(file_name, sheet_name=None, index_col=None)
df = pd.concat(df.values(),ignore_index=True)
target_column = "target"
source_column = "source"

df = df.rename(columns={"Intelas Ledger Name": target_column, "Source Ledger Name": source_column})
df_to_save = df.apply(lambda x: {'id':str(x.name), 'translation': {'source' : x[source_column], 'target' : x[target_column]} }, axis=1)
with open(save_json_name, 'w') as fout:
    json.dump(list(df_to_save.to_dict().values()) , fout)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
translator = pipeline("translation_source_to_target", model=model_name,max_length=30)
test_dataset = LedgerDataset(save_json_name,tokenizer)


In [ ]:
prefix = "translate source to target: "
res = []
for item in test_dataset : 
    text = prefix + item["translation"]["source"]
    target_text = item["translation"]["target"]
    pred = translator(text)[0]['translation_text']
    res.append({
        "text":text, 
        "target" : target_text,
        "pred" : pred, 
    })

In [ ]:
targets =[x["target"] for x in res]
predictions = [x["pred"] for x in res]

In [ ]:
sum =0 
failed= 0 
for i in range(len(targets)): 
    if targets[i] == predictions[i]:
        sum+=1
    else : 
        failed +=1 
        print(f"target : {targets[i]}\n prediction : {predictions[i]}")
print("accuracy : ")
print(sum/len(targets))
print(failed/len(targets))

In [ ]:
precision_recall_fscore_support(np.array(targets), np.array(predictions), average='weighted')